In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(r'C:\Users\91638\Downloads\Synthetic_Financial_datasets_log.csv\Synthetic_Financial_datasets_log.csv')

In [6]:
df = data

In [7]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
# Check for missing values
print(df.isnull().sum())

# Drop or fill missing values if necessary
df = df.dropna() 

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [9]:
# Identify columns with few unique values for one-hot encoding
columns_to_encode = ['type']  # Replace 'type' with the actual column name(s) you want to encode
df = pd.get_dummies(df, columns=columns_to_encode, drop_first=True)

In [10]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoding to high-cardinality columns
high_cardinality_columns = ['nameOrig', 'nameDest']  # Replace with actual column names
label_encoder = LabelEncoder()
for col in high_cardinality_columns:
    df[col] = label_encoder.fit_transform(df[col])

C:\Users\91638\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [11]:
df = df.drop(['nameOrig', 'nameDest'], axis=1)

In [12]:
# Convert numerical columns to more memory-efficient data types
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].astype('float32')

for col in df.select_dtypes(include=['int64']).columns:
    df[col] = df[col].astype('int32')

In [13]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop(columns=['isFraud'])  # Replace 'is_fraud' with the actual target column
y = df['isFraud']

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define the autoencoder architecture
input_dim = X_train_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(16, activation="relu")(input_layer)
encoder = Dense(8, activation="relu")(encoder)
encoder = Dense(4, activation="relu")(encoder)
decoder = Dense(8, activation="relu")(encoder)
decoder = Dense(16, activation="relu")(decoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
history = autoencoder.fit(X_train_scaled, X_train_scaled, 
                          epochs=50, 
                          batch_size=64, 
                          validation_data=(X_test_scaled, X_test_scaled),
                          verbose=1)

Epoch 1/50
79533/79533 [==============================] - 124s 2ms/step - loss: 0.7368 - val_loss: 0.7325
Epoch 2/50
79533/79533 [==============================] - 137s 2ms/step - loss: 0.7322 - val_loss: 0.7315
Epoch 3/50
79533/79533 [==============================] - 138s 2ms/step - loss: 0.7316 - val_loss: 0.7313
Epoch 4/50
79533/79533 [==============================] - 145s 2ms/step - loss: 0.7315 - val_loss: 0.7313
Epoch 5/50
79533/79533 [==============================] - 141s 2ms/step - loss: 0.7315 - val_loss: 0.7312
Epoch 6/50
79533/79533 [==============================] - 137s 2ms/step - loss: 0.7315 - val_loss: 0.7312
Epoch 7/50
79533/79533 [==============================] - 136s 2ms/step - loss: 0.7314 - val_loss: 0.7312
Epoch 8/50
79533/79533 [==============================] - 133s 2ms/step - loss: 0.7314 - val_loss: 0.7312
Epoch 9/50
79533/79533 [==============================] - 138s 2ms/step - loss: 0.7314 - val_loss: 0.7312
Epoch 10/50
79533/79533 [=====================

In [16]:
import numpy as np

# Get reconstruction loss for training data
reconstructions = autoencoder.predict(X_train_scaled)
train_loss = np.mean(np.power(X_train_scaled - reconstructions, 2), axis=1)

# Set a threshold based on training loss
threshold = np.percentile(train_loss, 95)

# Apply the threshold on test data
test_reconstructions = autoencoder.predict(X_test_scaled)
test_loss = np.mean(np.power(X_test_scaled - test_reconstructions, 2), axis=1)
y_pred = [1 if loss > threshold else 0 for loss in test_loss]  # 1 for fraud, 0 for normal

39767/39767 [==============================] - 52s 1ms/step


In [17]:
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc

# Calculate ROC-AUC
roc_score = roc_auc_score(y_test, y_pred)

# Calculate Precision, Recall, F1-Score
print(classification_report(y_test, y_pred))

# Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc = auc(recall, precision)

print(f"ROC-AUC Score: {roc_score}")
print(f"Precision-Recall AUC: {pr_auc}")

              precision    recall  f1-score   support

           0       1.00      0.95      0.97   1270904
           1       0.01      0.26      0.01      1620

    accuracy                           0.95   1272524
   macro avg       0.50      0.60      0.49   1272524
weighted avg       1.00      0.95      0.97   1272524

ROC-AUC Score: 0.6041237624343297
Precision-Recall AUC: 0.13276667057202088


In [18]:
# Assuming 'df' is the DataFrame that contains your dataset and 'is_fraud' is the target column.
X = df.drop(columns=['isFraud'])  # Replace 'is_fraud' with the actual target column

In [19]:
import numpy as np
import pandas as pd

# Example of a new transaction with the same feature columns as X
new_transaction = {
    'feature_1': 0.2,  # Replace 'feature_1' with actual feature names from the dataset
    'feature_2': 1500, 
    'feature_3': 0.5,  
    # Add values for each feature column present in the original dataset.
}

# Convert the dictionary to a DataFrame
new_transaction_df = pd.DataFrame([new_transaction])

# Align columns with the training set (in case some features were one-hot encoded or missing)
new_transaction_df = new_transaction_df.reindex(columns=X.columns, fill_value=0)

In [20]:
# Scale the new transaction data
new_transaction_scaled = scaler.transform(new_transaction_df)

In [21]:
# Get reconstruction loss for the new transaction
new_reconstruction = autoencoder.predict(new_transaction_scaled)
new_loss = np.mean(np.power(new_transaction_scaled - new_reconstruction, 2), axis=1)

# Apply the threshold to classify as fraud or not
new_prediction = [1 if loss > threshold else 0 for loss in new_loss]  # 1 for fraud, 0 for normal

# Output the prediction
print(f"Reconstruction loss: {new_loss[0]}")
print(f"Predicted label: {'Fraudulent' if new_prediction[0] == 1 else 'Normal'}")

1/1 [==============================] - 0s 31ms/step
Reconstruction loss: 0.41369692117507834
Predicted label: Normal
